In [2]:
# -*- coding: utf-8 -*-
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='--conf spark.sql.catalogImplementation=in-memory pyspark-shell'
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())


from pyspark.sql.types import *

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.3
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [ ]:
spark.stop()

In [3]:
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark import sql
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark.sql.functions as F
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.feature import Word2Vec
import pandas as pd

    Load the whole datasets

In [4]:
train_path = os.path.join('/labs/lab10data/', 'lab10_train.csv')
item_path = os.path.join('/labs/lab10data/', 'lab10_items.csv')
test_path = os.path.join('/labs/lab10data/', 'lab10_test.csv')
views_programmes = os.path.join('/labs/lab10data', 'lab10_views_programmes.csv')
res_path = os.path.join('/user/artem.moskalets/')

In [5]:
train_fields = [StructField('user_id', LongType(), True),
                StructField('item_id', LongType(), True),
               StructField('purchase', ByteType(), True)]
schema_train = StructType(train_fields)
df_train = spark.read.csv(header = 'true', schema = schema_train, path = train_path, sep=',')

In [6]:
df_train.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- item_id: long (nullable = true)
 |-- purchase: byte (nullable = true)



In [7]:
df_train.show()

+-------+-------+--------+
|user_id|item_id|purchase|
+-------+-------+--------+
|   1654|  74107|       0|
|   1654|  89249|       0|
|   1654|  99982|       0|
|   1654|  89901|       0|
|   1654| 100504|       0|
|   1654|  66187|       0|
|   1654|  84350|       0|
|   1654|  92854|       0|
|   1654|  72811|       0|
|   1654|  86876|       0|
|   1654| 102657|       0|
|   1654| 100482|       0|
|   1654|  89677|       0|
|   1654|  99419|       0|
|   1654|  66603|       0|
|   1654|   7363|       0|
|   1654|   1320|       0|
|   1654|  88892|       0|
|   1654|  66671|       0|
|   1654|  75925|       0|
+-------+-------+--------+
only showing top 20 rows



In [8]:
item_fields = [StructField('item_id', LongType(), True),
               StructField('channel_id', DoubleType(), True),
               StructField('datetime_availability_start', TimestampType(), True),
               StructField('datetime_availability_stop', TimestampType(), True),
               StructField('datetime_show_start', TimestampType(), True),
               StructField('datetime_show_stop', TimestampType(), True),
               StructField('content_type', ByteType(), True),
               StructField('title', StringType(), True),
               StructField('year', DoubleType(), True),
               StructField('genres', StringType(), True),
               StructField('region_id', FloatType(), True)]               
schema_item = StructType(item_fields)
df_item = spark.read.csv(header = 'true', schema = schema_item, path = item_path, sep='\t')

In [9]:
names = df_item. select('genres')
names.show()

+--------------------+
|              genres|
+--------------------+
|             Эротика|
|             Эротика|
|             Эротика|
|             Эротика|
|             Эротика|
|             Комедии|
|   Комедии,Мелодрамы|
|Ужасы,Триллеры,Др...|
|Ужасы,Комедии,Фан...|
|Комедии,Мелодрамы...|
|Детективы,Триллер...|
|Фантастика,Боевик...|
|Детективы,Триллер...|
|Детективы,Триллер...|
|Ужасы,Детективы,Т...|
|Комедии,Драмы,Зар...|
|Военные,Боевики,Наши|
|Приключения,Истор...|
|Военные,Драмы,Ист...|
|Комедии,Драмы,Мел...|
+--------------------+
only showing top 20 rows



In [10]:
names = names.withColumn('genres_s',F.split("genres",','))
names.printSchema()

root
 |-- genres: string (nullable = true)
 |-- genres_s: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [11]:
names = names.select(F.explode('genres_s').alias('genres_s')).groupby().agg(F.collect_set('genres_s'))
names_genres = names.rdd.flatMap(lambda x: x).collect()

In [12]:
for i in names_genres:
    for j in sorted(i):
        print (j)

 сказка
General
Анимация
Аниме
Арт-хаус
Артхаус
Биография
Боевик
Боевики
Вестерн
Видеоигры
Военные
Военный
Детективы
Детские
Детские песни
Для взрослых
Для всей семьи
Для детей
Для самых маленьких
Документальные
Документальный
Драма
Драмы
Западные мультфильмы
Зарубежные
Игры
Исторические
Исторический
Комедии
Комедия
Короткометражки
Короткометражные
Криминал
Кулинария
Мелодрама
Мелодрамы
Мистические
Музыкальные
Музыкальный
Мультсериалы
Мультфильм
Мультфильмы
Мультфильмы в 3D
Мюзиклы
Научная фантастика
Наши
О здоровье
Охота и рыбалка
Передачи
Познавательные
Полнометражные
Приключение
Приключения
Про животных
Прочие
Развивающие
Развлекательные
Реалити-шоу
Романтические
Русские
Русские мультфильмы
Семейные
Семейный
Сериалы
Сказки
Советские
Советское кино
Союзмультфильм
Спорт
Спортивные
Триллер
Триллеры
Ужасы
Фантастика
Фантастические
Фильмы
Фильмы в 3D
Фильмы-спектакли
Фэнтези
Хочу всё знать
Экранизации
Эротика
Юмористические


In [13]:
dictionary = {u'арт-хаус': u'артхаус',             
u'военный': u'военные',\
u'боевики': u'боевик',\
u'вестерн': u'вестерн',\
u'видеоигры': u'видеоигры, игры',\
u'детективы':u'детективы',\
u'документальный': u'документальные',\
u'драмы': u'драма',\
u'исторический': u'исторические',\
u'комедия': u'комедии',\
u'короткометражные':u'короткометражки',
u'мелодрама': u'мелодрамы',\
u'мистические': u'мистические',\
u'музыкальные': u'музыкальный',\
u'мьюзиклы': u'мьюзиклы,музыкальный',\
u'приключение': u'приключения',\
u'романтические': u'мелодрамы',\
u'семейный': u'семейные',\
u'для всей семьи': u'семейные',\
u'советское кино': u'советские, русские',\
u'наши': u'русские',\
u'спортивные': u'спорт',\
u'ужасы': u'триллер, ужасы',\
u'фантастика': u'фантастические',\
u'фильмы в 3d': u'фильмы,3d',\
u'фэнтези': u'фэнтези,фантастические',\
u'научная фантастика': u'научные,фантастические',\
u'хочу всё знать': u'познавательные, развивающие',\
u'познавательные': u'познавательные, развивающие',\
u'детские песни': u'детские,песни',\
u'для детей': u'детские',\
u'для самых маленьких': u'детские',\
u'западные мультфильмы': u'детские, мультфильм,западные',\
u'западные детские' : u'западные, детские',
u'русские детские' : u'русские, детские',             
u'зарубежные': u'зарубежные,западные',
u'мультсериалы': u'детские, мультфильм',\
u'мультфильмы': u'детские,мультфильм',\
u'мультфильм в 3d': u'детские,мультфильм, 3d',\
u'русские мультфильмы': u'детские,мультфильм,русские',\
u'развивающие': u'детские,развивающие',\
u'сказки': u'детские,сказки',\
u'сказка': u'детские,сказки',\
u'триллеры': u'триллер',\
u'союзмультфильм': u'детские,мультфильм',\
u'анимация': u'детские,мультфильм',\
u'эротика': u'эротика, для взрослых'}


In [14]:
test_fields = [StructField('user_id', LongType(), True),
              StructField('item_id', LongType(), True),
              StructField('purchase', ByteType(), True)]
schema_test = StructType(test_fields)
df_test = spark.read.csv(header = 'true', schema = schema_test, path = test_path, sep=',')

In [15]:
df_test.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- item_id: long (nullable = true)
 |-- purchase: byte (nullable = true)



In [16]:
views_programmes_fields = [StructField('user_id', LongType(), True),
                           StructField('item_id', LongType(), True),
                           StructField('ts_start', LongType(), True),
                           StructField('ts_end', LongType(), True),
                           StructField('item_type', StringType(), True)]
schema_views_programmes = StructType(views_programmes_fields)
df_views_programmes = spark.read.csv(header = 'true', schema = schema_views_programmes, path = views_programmes, sep=',')

In [17]:
df_views_programmes.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- item_id: long (nullable = true)
 |-- ts_start: long (nullable = true)
 |-- ts_end: long (nullable = true)
 |-- item_type: string (nullable = true)



    Feature engineering

In [18]:
df_features = df_train.join(df_item, df_train.item_id == df_item.item_id, "inner").drop(df_item.item_id)

In [19]:
df_features = df_features.select('user_id','item_id','purchase','year','genres')

In [20]:
df_features = df_features.select('*', coalesce(df_features["year"], lit(0.0)).alias('year_0'))


In [21]:
df_features = df_features.drop("year").select('*', col('year_0').alias("year")).drop('year_0')

In [22]:
df_features = df_features.fillna('noname')

In [23]:
def gen(films):
    lst =  films.split(',')
    res = ''
    for l in lst:
        l = l.lower().strip()
        for key in list(dictionary.keys()):
            l = l.replace(key, dictionary[key])
        res = res + ','+ l +','
    return res
gen =  udf(gen,StringType())  

In [24]:
df_features = df_features.withColumn('Words', gen(df_features.genres))

In [25]:
df_features.select(df_features.Words).show()

+--------------------+
|               Words|
+--------------------+
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
+--------------------+
only showing top 20 rows



In [26]:
def un(one, two, three, four):
    return str(one)+','+str(two) +','+str(three)+','+(four)
un = udf(un,StringType())
df_features = df_features.withColumn('all_features', un('user_id','item_id','year','words'))

In [27]:
df_features.select('all_features').show(truncate = False)

+------------------------------------------------------------------------+
|all_features                                                            |
+------------------------------------------------------------------------+
|1654,3764,2013.0,,триллер, ужасы,,триллер,,драма,,зарубежные,западные,  |
|510087,3764,2013.0,,триллер, ужасы,,триллер,,драма,,зарубежные,западные,|
|522798,3764,2013.0,,триллер, ужасы,,триллер,,драма,,зарубежные,западные,|
|529632,3764,2013.0,,триллер, ужасы,,триллер,,драма,,зарубежные,западные,|
|566758,3764,2013.0,,триллер, ужасы,,триллер,,драма,,зарубежные,западные,|
|613775,3764,2013.0,,триллер, ужасы,,триллер,,драма,,зарубежные,западные,|
|619378,3764,2013.0,,триллер, ужасы,,триллер,,драма,,зарубежные,западные,|
|625638,3764,2013.0,,триллер, ужасы,,триллер,,драма,,зарубежные,западные,|
|632495,3764,2013.0,,триллер, ужасы,,триллер,,драма,,зарубежные,западные,|
|639612,3764,2013.0,,триллер, ужасы,,триллер,,драма,,зарубежные,западные,|
|651811,3764,2013.0,,трил

In [28]:
df_features.select(df_features.Words).count()

5032624

In [29]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, RegexTokenizer

tokenizer = RegexTokenizer(inputCol="all_features", 
                           outputCol="lst", 
                           gaps=False, 
                           pattern=r'[a-zA-Zа-яА-Яё0-9_\-]{2,}')
df_features = tokenizer.transform(df_features)

In [30]:
df_features.select('lst').show(truncate = False)

+--------------------------------------------------------------------------+
|lst                                                                       |
+--------------------------------------------------------------------------+
|[836538, 3764, 2013, триллер, ужасы, триллер, драма, зарубежные, западные]|
|[836889, 3764, 2013, триллер, ужасы, триллер, драма, зарубежные, западные]|
|[836923, 3764, 2013, триллер, ужасы, триллер, драма, зарубежные, западные]|
|[836953, 3764, 2013, триллер, ужасы, триллер, драма, зарубежные, западные]|
|[837038, 3764, 2013, триллер, ужасы, триллер, драма, зарубежные, западные]|
|[837166, 3764, 2013, триллер, ужасы, триллер, драма, зарубежные, западные]|
|[837219, 3764, 2013, триллер, ужасы, триллер, драма, зарубежные, западные]|
|[837578, 3764, 2013, триллер, ужасы, триллер, драма, зарубежные, западные]|
|[837780, 3764, 2013, триллер, ужасы, триллер, драма, зарубежные, западные]|
|[837850, 3764, 2013, триллер, ужасы, триллер, драма, зарубежные, западные]|

In [31]:
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.linalg import Vectors

In [32]:
cv = CountVectorizer(inputCol= "lst", outputCol="features")
model_cv = cv.fit(df_features)

In [33]:
df_features = model_cv.transform(df_features)

In [34]:
df_features.show()

+-------+-------+--------+--------------------+------+--------------------+--------------------+--------------------+--------------------+
|user_id|item_id|purchase|              genres|  year|               Words|        all_features|                 lst|            features|
+-------+-------+--------+--------------------+------+--------------------+--------------------+--------------------+--------------------+
| 909525|   3764|       0|Ужасы,Триллеры,Др...|2013.0|,триллер, ужасы,,...|909525,3764,2013....|[909525, 3764, 20...|(5783,[0,1,4,5,12...|
| 909582|   3764|       0|Ужасы,Триллеры,Др...|2013.0|,триллер, ужасы,,...|909582,3764,2013....|[909582, 3764, 20...|(5783,[0,1,4,5,12...|
| 909603|   3764|       0|Ужасы,Триллеры,Др...|2013.0|,триллер, ужасы,,...|909603,3764,2013....|[909603, 3764, 20...|(5783,[0,1,4,5,12...|
| 909741|   3764|       0|Ужасы,Триллеры,Др...|2013.0|,триллер, ужасы,,...|909741,3764,2013....|[909741, 3764, 20...|(5783,[0,1,4,5,12...|
| 909745|   3764|       0|У

In [35]:
df_features = df_features.drop('genres','Words','user_id','item_id','all_features','genres','year','lst')

In [36]:
df_features.show()

+--------+--------------------+
|purchase|            features|
+--------+--------------------+
|       0|(5783,[0,1,4,5,12...|
|       0|(5783,[0,1,4,5,12...|
|       0|(5783,[0,1,4,5,12...|
|       0|(5783,[0,1,4,5,12...|
|       0|(5783,[0,1,4,5,12...|
|       0|(5783,[0,1,4,5,12...|
|       0|(5783,[0,1,4,5,12...|
|       0|(5783,[0,1,4,5,12...|
|       0|(5783,[0,1,4,5,12...|
|       0|(5783,[0,1,4,5,12...|
|       0|(5783,[0,1,4,5,12...|
|       0|(5783,[0,1,4,5,12...|
|       0|(5783,[0,1,4,5,12...|
|       0|(5783,[0,1,4,5,12...|
|       0|(5783,[0,1,4,5,12...|
|       0|(5783,[0,1,4,5,12...|
|       0|(5783,[0,1,4,5,12...|
|       0|(5783,[0,1,4,5,12...|
|       0|(5783,[0,1,4,5,12...|
|       0|(5783,[0,1,4,5,12...|
+--------+--------------------+
only showing top 20 rows



In [38]:
# Index labels, adding metadata to the label column.
from pyspark.ml.feature import StringIndexer, VectorIndexer

# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="purchase", outputCol="indexedLabel").fit(df_features)
# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
VectorIndexer(inputCol="features", outputCol="indexedFeatures").fit(df_features)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = df_features.randomSplit([0.7, 0.3])

In [61]:
from pyspark.ml.classification import RandomForestClassifier, RandomForestClassificationModel

# Train a RandomForest model.
rf = RandomForestClassifier(numTrees = 800, maxDepth = 12, labelCol="purchase", featuresCol="features")

In [62]:
# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf])

In [63]:
# Train model.  This also runs the indexers.
model = rf.fit(df_features)

In [64]:
model.save('RF_lab10_3.sav')

In [65]:
model = RandomForestClassificationModel.load('RF_lab10_3.sav')

In [66]:
df_test.show()

+-------+-------+--------------------+
|user_id|item_id|            features|
+-------+-------+--------------------+
| 517612|   3764|(5783,[0,1,4,5,12...|
| 520446|   3764|(5783,[0,1,4,5,12...|
| 523860|   3764|(5783,[0,1,4,5,12...|
| 556825|   3764|(5783,[0,1,4,5,12...|
| 566701|   3764|(5783,[0,1,4,5,12...|
| 575248|   3764|(5783,[0,1,4,5,12...|
| 588378|   3764|(5783,[0,1,4,5,12...|
| 625678|   3764|(5783,[0,1,4,5,12...|
| 627053|   3764|(5783,[0,1,4,5,12...|
| 636572|   3764|(5783,[0,1,4,5,12...|
| 642397|   3764|(5783,[0,1,4,5,12...|
| 668112|   3764|(5783,[0,1,4,5,12...|
| 703514|   3764|(5783,[0,1,4,5,12...|
| 711308|   3764|(5783,[0,1,4,5,12...|
| 736010|   3764|(5783,[0,1,4,5,12...|
| 739215|   3764|(5783,[0,1,4,5,12...|
| 740405|   3764|(5783,[0,1,4,5,12...|
| 742324|   3764|(5783,[0,1,4,5,12...|
| 742986|   3764|(5783,[0,1,4,5,12...|
| 746959|   3764|(5783,[0,1,4,5,12...|
+-------+-------+--------------------+
only showing top 20 rows



In [67]:
df_test = spark.read.csv(header = 'true', schema = schema_test, path = test_path, sep=',')
df_test = df_test.join(df_item, df_test.item_id == df_item.item_id, "inner").drop(df_item.item_id)
df_test = df_test.select('user_id','item_id','year','genres')
df_test = df_test.select('*', coalesce(df_test["year"], lit(0.0)).alias('year_0'))
df_test = df_test.drop("year").select('*', col('year_0').alias("year")).drop('year_0')
df_test = df_test.fillna('noname')
df_test = df_test.withColumn('Words', gen(df_test.genres))
df_test = df_test.withColumn('all_features', un('user_id','item_id','year','words'))
df_test = tokenizer.transform(df_test)
df_test = model_cv.transform(df_test)
df_test = df_test.drop('genres','Words','all_features','genres','year','lst','purchase')

In [68]:
df_test.show()

+-------+-------+--------------------+
|user_id|item_id|            features|
+-------+-------+--------------------+
| 874011|   3764|(5783,[0,1,4,5,12...|
| 874417|   3764|(5783,[0,1,4,5,12...|
| 874709|   3764|(5783,[0,1,4,5,12...|
| 875202|   3764|(5783,[0,1,4,5,12...|
| 875222|   3764|(5783,[0,1,4,5,12...|
| 875274|   3764|(5783,[0,1,4,5,12...|
| 875552|   3764|(5783,[0,1,4,5,12...|
| 875580|   3764|(5783,[0,1,4,5,12...|
| 875609|   3764|(5783,[0,1,4,5,12...|
| 875996|   3764|(5783,[0,1,4,5,12...|
| 876215|   3764|(5783,[0,1,4,5,12...|
| 876221|   3764|(5783,[0,1,4,5,12...|
| 876234|   3764|(5783,[0,1,4,5,12...|
| 876263|   3764|(5783,[0,1,4,5,12...|
| 876350|   3764|(5783,[0,1,4,5,12...|
| 876426|   3764|(5783,[0,1,4,5,12...|
| 876455|   3764|(5783,[0,1,4,5,12...|
| 876758|   3764|(5783,[0,1,4,5,12...|
| 876896|   3764|(5783,[0,1,4,5,12...|
| 877001|   3764|(5783,[0,1,4,5,12...|
+-------+-------+--------------------+
only showing top 20 rows



In [69]:
df_test.count()

2156840

In [70]:
2156840

2156840

In [71]:
df_test.show()

+-------+-------+--------------------+
|user_id|item_id|            features|
+-------+-------+--------------------+
| 517612|   3764|(5783,[0,1,4,5,12...|
| 520446|   3764|(5783,[0,1,4,5,12...|
| 523860|   3764|(5783,[0,1,4,5,12...|
| 556825|   3764|(5783,[0,1,4,5,12...|
| 566701|   3764|(5783,[0,1,4,5,12...|
| 575248|   3764|(5783,[0,1,4,5,12...|
| 588378|   3764|(5783,[0,1,4,5,12...|
| 625678|   3764|(5783,[0,1,4,5,12...|
| 627053|   3764|(5783,[0,1,4,5,12...|
| 636572|   3764|(5783,[0,1,4,5,12...|
| 642397|   3764|(5783,[0,1,4,5,12...|
| 668112|   3764|(5783,[0,1,4,5,12...|
| 703514|   3764|(5783,[0,1,4,5,12...|
| 711308|   3764|(5783,[0,1,4,5,12...|
| 736010|   3764|(5783,[0,1,4,5,12...|
| 739215|   3764|(5783,[0,1,4,5,12...|
| 740405|   3764|(5783,[0,1,4,5,12...|
| 742324|   3764|(5783,[0,1,4,5,12...|
| 742986|   3764|(5783,[0,1,4,5,12...|
| 746959|   3764|(5783,[0,1,4,5,12...|
+-------+-------+--------------------+
only showing top 20 rows



In [72]:
predictions = model.transform(df_test)

In [73]:
def proba1(vector):
    from pyspark.ml.linalg import Vectors
    res = float(vector[1])
    return res
proba1 = udf(proba1, FloatType())

In [74]:
predictions1 = predictions.select('*', proba1('probability').alias('purchase'))

In [75]:
predictions1.show()

+-------+-------+--------------------+--------------------+--------------------+----------+------------+
|user_id|item_id|            features|       rawPrediction|         probability|prediction|    purchase|
+-------+-------+--------------------+--------------------+--------------------+----------+------------+
| 874011|   3764|(5783,[0,1,4,5,12...|[798.359971866320...|[0.99794996483290...|       0.0|0.0020500352|
| 874417|   3764|(5783,[0,1,4,5,12...|[798.364479377077...|[0.99795559922134...|       0.0|0.0020444007|
| 874709|   3764|(5783,[0,1,4,5,12...|[798.364479377077...|[0.99795559922134...|       0.0|0.0020444007|
| 875202|   3764|(5783,[0,1,4,5,12...|[798.364479377077...|[0.99795559922134...|       0.0|0.0020444007|
| 875222|   3764|(5783,[0,1,4,5,12...|[798.364479377077...|[0.99795559922134...|       0.0|0.0020444007|
| 875274|   3764|(5783,[0,1,4,5,12...|[798.364479377077...|[0.99795559922134...|       0.0|0.0020444007|
| 875552|   3764|(5783,[0,1,4,5,12...|[798.364479377077

In [76]:
predictions1.count()

2156840

In [77]:
Res = predictions1.select("user_id","item_id","purchase").\
                    sort("user_id","item_id")

In [78]:
Res.coalesce(1).write.mode("overwrite").option("header", "true").csv("lab10_3.csv")

In [79]:
!hadoop fs -getmerge lab10_3.csv lab10s.csv